# TME 9: Merge
> Consignes: le fichier TME8_Sujet.ipynb est à déposer sur le site Moodle de l'UE https://moodle-sciences.upmc.fr/moodle-2018/course/view.php?id=4650. Si vous êtes en binôme, renommez-le en TME8_nom1_nom2.ipynb.

N'oubliez pas de sauvegarder fréquemment votre notebook !!

## Données et codes utiles au TME

In [9]:
from PIL import Image
from pylab import *

### Données
img_test = full((64,64),150,dtype=uint8)
img_test[32:48,16:16+32] = 100
img_test[8:24,40:56] = 50
angio = array(Image.open('images/angiogra.png'))
cam = array(Image.open('images/cameraman.png'))
muscle = array(Image.open('images/muscle.png'))
prisme = array(Image.open('images/prisme.png'))
seiche = array(Image.open('images/seiche.png'))

### Fonction d'affichage
def affichage_14(affichages,titres=None):
    """ list[Array]*list[str] -> NoneType
    Dffectue entre 1 et 4 affichages avec leurs titres, pour des images ou courbes
    
    Paramètres :
      - liste des affichages (entre 1 et 4)
      - liste des titres (entre 1 et 4, autant que de affichages), optionnelle    
    """
    
    if not type(affichages) == type([]):
        affichages = [affichages]
        
    if titres is None:
        titres = ['',]*len(affichages)
        
    if not type(titres) == type([]):
        titres = [titres]
    
    nb_affichages = len(affichages)
    if nb_affichages >4 or nb_affichages < 1 :
        raise ValueError('affichage_14 nécéssite 1 à 4 entrées en paramètre')        
        
    if nb_affichages != len(titres):
        raise ValueError('affichage_14 nécéssite autant de titres que d\'affichages')
        
    courbes = False
    for i in range(0,nb_affichages):
        s = subplot(101+10*nb_affichages+i)
        s.set_title(titres[i])
        if len(affichages[i].shape)==2 and affichages[i].shape[0] > 1 and affichages[i].shape[1] > 1:
            # on affiche une image
            s.imshow(affichages[i], cmap="gray",interpolation='nearest', aspect='equal')
        else :
            # il s'agit d'une seule ligne, à afficher comme une courbe
            plot(affichages[i])
            courbes=True

    agrandissement_h = nb_affichages
    agrandissement_v = nb_affichages*2 if courbes else nb_affichages
    params = gcf()
    plSize = params.get_size_inches()
    params.set_size_inches( (plSize[0]*agrandissement_v, plSize[1]*agrandissement_h) )
    show()



Recopier ci-dessous le code des fonctions ``split()``, ``predsplit1()`` et ``splitprof()``
écrites au TME 8:

In [47]:
def split(I,R,pred,*args):
    """ Array*Bloc*(Array*Bloc*...->bool)*... -> 4-aire
        Découpage de I en arbre 4-aire sur un critère d'écart-type minimal
    """

    x,y,n,m=R
    if( predsplit1(I,R,*args) and n>2 and m>2):
        NO=split(I,(x,y,n//2,m//2),pred,*args)
        NE=split(I,(x,y+m//2,n//2,m//2),pred,*args)
        SO=split(I,(x+n//2,y,n//2,m//2),pred,*args)
        SE=split(I,(x+n//2,y+m//2,n//2,m//2),pred,*args)
        return [NO,NE,SO,SE]
    
    else:
        return [R]
    

def predsplit1(I,R,*args):
    """ Array*Bloc*... -> bool
    """ 
    x,y,n,m=R
    return I[x:x+n,y:y+m].std()>args[0]

def splitprof(L):
    """ 4-aire -> list[Bloc]"""
    if(len(L)==1):
        return L[0]
    else:
        for l in L:
            splitprof(l)


## Exercice 1
### Question 1
Modifier la fonction ``splitprof(L)`` pour en faire une fonction ``splitprofstat(L,I)`` qui rend une liste de feuilles de l'arbre 4-aire ``L`` et telle que chaque élément de cette liste, de type ``Bloc``, est enrichi de trois scalaires, le premier étant la taille, le second la moyenne et la troisième la variance des valeurs de pixels du bloc dans l'image ``I``. Cette fonction rend alors une liste dont les éléments ont pour type formel:
```python
type Blocstat = tuple[int**4,int,float**2]
```
Les quatre premières valeurs sont celles du type ``Bloc``, la cinquième est la taille du bloc (en nombre de pixels) et les deux dernières correspondent à la moyenne et la variance calculées sur la région.


In [56]:
# type Blocstat = tuple[int**4,int,float**2]

def splitprofstats(L,I):
    """ 4-aire*Array -> list[Blocstat] """
    """ 4-aire -> list[Bloc]"""
    if(len(L)==1):
        x,y,n,m=L[0]
        return (L[0],((n-x)*(m-y)),mean((I[x:x+n,y:y+m])),var((I[x:x+n,y:y+m])))
    else:
        for l in L:
            splitprofstats(l,I)


### Question 2
Dans la suite du TME, on considère le type formel:
```python
type Region = list[Blocstat]
```
Une région, comme vu au TME, est donc une liste de blocs. Écrire le prédicat de fusion ``predmerge(b,R,*args)`` vu en TD. Cette fonction retourne ```True``` si le bloc ``b`` doit fusionner dans la région ``R``. Si telle est le cas, alors le premier élément de la liste ``R`` voit ses statistiques modifiées: elles décrivent celles de l'ensemble des blocs de la région ainsi fusionnée.

In [53]:
def predmerge(b,R,seuil):
    """ Blocstat*Region*... -> bool 
        Si fusion, R[0] est modifié
    """
    x1,y1,w1,h1,n1,u1,s1_2=b
    x2,y2,w2,h2,n2,u2,s2_2=R[0]
    n=n1+n2
    u=(((n1*u1)+(n2*u2)))/n
    s_2=((n1*s1_2)+(n2*s2_2)+(n1*u1**2)+(n2*u2**2))/n-u**2
    if s_2<seuil:
        R[0]=x2,y2,w2,h2,b,u,s_2
        return true
    return false


### Question 3
Implémenter les fonctions ```estdans()```, ```sontvoisins()```, ```voisins()``` et finalement ```merge()``` vues en TD.

In [58]:
def estdans(x,y,a):
    """ int*int*Blocstat -> bool 
        Teste si (x,y) est dans a 
    """
    return x>=a.x and y>= a.y and x<(a.x+a.w) and y<(a.y+a.h)

def sontvoisins(a,b):
    """ Blocstat**2 -> bool
        Teste si a et b sont voisins
    """
    xa,ya,wa,ha=a
    xb,yb,wb,h=b
    if (wa>wb) and (ha>hb):
        (a,b)=(b,a)
    xcentre=xa+(wa//2)
    ycentre=ya+(ha//2)
    return (estdans(xcentre-wa,ycentre,b)
           or estdans(xcentre+wa,ycentre,b)
           or estdans(xcentre,ycentre-ha,b)
           or estdans(xcentre,ycentre+ha,b))
def voisins(b,K):
    """ Blocstat*list[Blocstat]->list[Blocstat]
        retourne la liste des elements de K voisins de b
    """
    N=[]
    for n in k:
        if(sontvoisins(b,n)):
            N.append(n)
    return n;

# type Region = list[Blocstat]

def merge(L,I,pred,*args):
    """ 4-aire*Array*(Blocstat*Region*...->bool) -> list[Region]
        Fusionne les feuilles de S en une liste de régions
    """
    K=[]
    for x,y,n,m in L:
        K.append(x,y,n,m,n*m,u,s_2)
    L=[]
    while(len(h)>0):
        b=K[0]
        K=K[1]
        R=[b]
        N=voisins(b,K)
        while(len[N]>0):
            bi=N[0]
            N=N[1:]
            if(predmerge(bi,R,seuil)):
                K.remove(bi)
                R.append(bi)
                for j in voisins (bi,K):
                    if not j in N:
                        N.append(j)
        L.append(R)
    return L


### Question 4
Faire la mise au point des fonctions précédentes à l'aide de l'image ``img_test``.

In [59]:
Bloc = (0,0,len(img_test)-1,len(img_test)-1)
pred=predsplit1(img_test,Bloc,1)
merge(splitprofstats(split(img_test,Bloc,pred,1),img_test),img_test,pred)


TypeError: 'NoneType' object is not iterable

### Question 5
Écrire une fonction ``regions(LR,shape)`` qui prend une liste de régions telle que rendu par la fonction ``merge()`` et rend une image des régions. Chaque région sera colorée avec le niveau de gris correspondant à la moyenne de la région. Le paramètre ``shape`` donne la taille de l'image à produire. Tester la fonction sur l'exemple précédent.

In [ ]:
def regions(LR,shape):
    """ list[Region] -> Array """
    buf = zeros(shape,dtype=uint8)
    for R in LR:
        x,y,n,m,s,mu,v=R[0]
        for b in R:
            x,y,n,m,s,t,v=b
            buf[x:x+n,y:y+m] = int(mu)
    return buf


## Exercice 2: expérimentation
### Question 1:
Tester la fonction ``merge()`` sur les images ``angio``, ``cam``, ``muscle``, ``prisme`` et ``seiche``.

In [11]:
L=()
mean((4,5,5))

4.666666666666667

### Question 2 (bonus):
Utiliser un critère de découpage alternatif, par exemple celui proposé en cours: on fusionne qui la variance de la région fusionné est plus petite que celle de la région non fusionnée. Tester sur les images.

### Question 3 (bonus):
La fusion dépend de la façon dont on visite les régions. On peut alors faire un tri des feuilles de l'arbre 4-aire, par exemple, des blocs les plus gros aux plus petits, ou le contraire (utiliser la fonction Python ``sorted()``). La même question peut se poser lorsqu'on calcule l'ensemble des voisins de la région que l'on fusionne. Faut-il les trier ? Si oui, selon quel critère ? leur taille ? leur proximité ? Evidemment il n'y a pas de réponse universelle mais adaptée à chaque type de problème. Faites quelques tests pour voir l'influence de ces tris sur le résultat de la fusion.